In [1]:
import pandas as pd
from model.op import FleetSizeOptimizer
import numpy as np

In [5]:
w = FleetSizeOptimizer(flight_time=np.array([[0, 10],[10, 0]]).astype(int), 
                       energy_consumption=np.array([[0, 16.25],[16.25, 0]]).astype(int), 
                       schedule='icrat_wind/demand/schedule_500pax_5min_0206.csv')
w.optimize(output_path=f'rl_sample/500pax_0206', verbose=True, optimality_gap=0.05)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-29
Set parameter Threads to value 2
Set parameter MIPGap to value 0.05
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 2 threads

Optimize a model with 21490 rows, 345708 columns and 678142 nonzeros
Model fingerprint: 0x163ce049
Variable types: 0 continuous, 345708 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-07, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 2239 rows and 31763 columns
Presolve time: 0.58s
Presolved: 19251 rows, 313945 columns, 636585 nonzeros
Variable types: 0 continuous, 313945 integer (10 binary)
Deterministic concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.33s

Ba

In [ ]:
w.parse_result('rl_sample/500pax_0206.txt')
w.calculate_aircraft_states()
w.get_summary_statistics(np.array([[0, 10],[10, 0]]).astype(int))

In [9]:
pax_arrival_time = pd.read_csv('input/icrat_wind/demand/pax_arrival_times_500pax_5min_0206.csv')
schedule = pd.read_csv('input/icrat_wind/demand/schedule_500pax_5min_0206.csv')
num_pax = pd.read_csv('input/icrat_wind/demand/num_pax_500pax_5min_0206.csv')

In [56]:
def get_optimized_waiting_time_statistics(pax_arrival_time, schedule, num_pax, return_num=False):
    schedule['schedule'] = np.ceil(schedule['schedule']/5)*5

    schedule_arr = schedule[schedule['od'] == 'LAX_DTLA']['schedule'].to_numpy()
    pax_arrival_arr = pax_arrival_time[pax_arrival_time['origin_vertiport_id'] == 'LAX']['passenger_arrival_time'].to_numpy()
    num_pax_arr = num_pax[num_pax['od'] == 'LAX_DTLA']['num_pax'].to_numpy()

    departure_time = np.zeros(0)
    for i in range(len(schedule_arr)):
        departure_time = np.append(departure_time, np.repeat(schedule_arr[i], num_pax_arr[i]))
    waiting_time = departure_time * 60 - pax_arrival_arr

    schedule_arr = schedule[schedule['od'] == 'DTLA_LAX']['schedule'].to_numpy()
    pax_arrival_arr = pax_arrival_time[pax_arrival_time['origin_vertiport_id'] == 'DTLA']['passenger_arrival_time'].to_numpy()
    num_pax_arr = num_pax[num_pax['od'] == 'DTLA_LAX']['num_pax'].to_numpy()

    departure_time = np.zeros(0)
    for i in range(len(schedule_arr)):
        departure_time = np.append(departure_time, np.repeat(schedule_arr[i], num_pax_arr[i]))
    waiting_time2 = departure_time * 60 - pax_arrival_arr

    waiting_time = np.append(waiting_time, waiting_time2)/60
    if return_num:
        return waiting_time.mean(), waiting_time.std(), waiting_time.max(), waiting_time.min(), len(waiting_time)
    else:
        print(f'Mean waiting time: {waiting_time.mean()}')
        print(f'Standard deviation of waiting time: {waiting_time.std()}')
        print(f'Maximum waiting time: {waiting_time.max()}')
        print(f'Minimum waiting time: {waiting_time.min()}')

In [59]:
print('500pax_5min_0206')
print('----------------')
get_optimized_waiting_time_statistics(pax_arrival_time, schedule, num_pax)

500pax_5min_0206
----------------
Mean waiting time: 5.0287361769352295
Standard deviation of waiting time: 2.395215013069476
Maximum waiting time: 10.0
Minimum waiting time: 0.0
